In [1]:
import pandas as pd
import numpy as np
import os
import string
import sys
import re
import datetime as dt
# now = dt.datetime.now().strftime('%y%m%d_%H%M%S')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
# null/입력 값 검색 로직

def nll_chk(data):
    cnt_chk = len(data)

    chk_nll_list = []
    chk_nll_cnt_list = []
    # null값 체크(개수 파악)
    for i,j in zip(range(len(data.columns)), data.columns):
        indv_cnt = data.loc[data[j] != '', j]
        if len(data[j]) != len(indv_cnt):
            print(j + ": " + str(len(indv_cnt)))
            chk_nll_list.append(j)
            chk_nll_cnt_list.append(len(indv_cnt) - cnt_chk)
    print("================================")
    if len(chk_nll_list) == 0:
        print("모든 값이 입력되었습니다.")
    elif len(chk_nll_list) > 1 :
        print("null이 포함된 컬럼이 탐지되었습니다.")
        print(chk_nll_list)
        print("================================")

    globals()["cnt_chk_out"] = pd.DataFrame(zip(chk_nll_list, chk_nll_cnt_list), columns = ["NULL포함컬럼명", "NULL개수"])
    print(cnt_chk_out)
    
    print("================================")
    print("null 인덱스를 임시 보관합니다.")
    print("================================")
    nll_chk_row = cnt_chk_out.iloc[:,0].to_list()
    for i, j in zip(range(len(nll_chk_row)), nll_chk_row):
        globals()["null_{}".format(i)] = data.loc[data[j] == '', j].index
        print("널인덱스보관_" + str(j) + "(변수명) -> null_{}".format(i))
        
# 스페이스 제거 함수
def rmv_spce(data, opt, *col):
    if opt == 'all': 
        for i in data.columns:
            data[i] = data[i].str.replace(" ", "")
            print(str(i) + ":\t 스페이스 제거 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data[col] = data[col].str.replace(" ", "")
        print(col + ":\t 스페이스 제거 성공")
#         print(type(col))

# 불필요 컬럼 빼기 - 이름으로 빼기
def brief_col(data, excpt_col_list, name):
    col_list = data.columns
    selected_col_list = [x for x in col_list if x not in excpt_col_list]
    print(selected_col_list)
    globals()["{}".format(name)] = data[selected_col_list]
    
# 불필요 컬럼 빼기 - 순서로 빼기 
# tmp_col_list = [5,6,7,8]
# tmp_tbl_list = [2,3,4,5]
def brief_col_num(data, excpt_col_list, name):
    col_list = data.columns
    col_list_num = [*range(len(col_list))]
    selected_col_list_num = [x for x in col_list_num if x not in excpt_col_list]
    selected_col = col_list[selected_col_list_num]
    print(selected_col)
    globals()["{}".format(name)] = data[selected_col]

# 컬럼명(순서) 선택 및 변경
def cor_col(data, select_col, modify_col, name):
    # 컬럼명(순서) 선택
    globals()["{}".format(name)] = data[select_col]
    # 컬렴명 변경
    globals()["{}".format(name)].columns = modify_col
    
# null 공백처리 함수
def nll_to_blnk(data, opt, *col):
    if opt == 'all':
        print("[모든 컬럼 Null 공백 처리]")
        for i in data.columns:
            data[i] = data[i].fillna("")
            print(str(i) + ":\t Null 공백 처리 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data[col] = data[col].fillna("")
        print(col + ":\t  공백 처리 성공")

# 컬럼 고유값 수 비교
def cmpar(data, col1, col2):
    crtr_1 = len(data[col1].unique()) - 1
    crtr_2 = len(data[col2].unique()) - 1
    if crtr_1 == crtr_2:
        print('[정상] 용어명과 분리된용어명의 고유값 수가 같습니다.= ' + str(crtr_1) + " vs " + str(crtr_2))
    else:
        print('[오류] 용어명과 분리된용어명의 고유값 수가 다릅니다.= ' + str(crtr_1) + " vs " + str(crtr_2))
        
# 공백 null처리 함수
def blank_to_nll(data, opt, *col):
    if opt == 'all':
        print("[모든 컬럼 공백 Null 처리]")
        for i in data.columns:
            data.loc[data[i] == '', i] = None
            print(str(i) + ":\t Null 처리 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data.loc[data[col] == '', col] = None
        print(col + ":\t  Null 처리 성공")
        
def row_nll_rmv(data, name):
    ### 값이 한개라도 있을 경우만 카운트 ###

    data_cnt = len(data)
    notna_list = []
    for i in range(data_cnt):    
        # row 기준 널 체크(값이 한개라도 있을 경우 포함)
        if data.iloc[i].notna().sum() > 1:
            notna_list.append(i)
    globals()["{}".format(name)] = globals()["{}".format(data)].loc[notna_list]
    print(notna_list)
    print(data.shape)
    
# blank row 삭제
def blnk_row_rmv(data, opt, *col):
    if opt == 'all':
        data_cnt = len(data)
        notna_list = []
        for i in range(data_cnt):
            if (data.iloc[i,:] != '').sum() > 1:
                notna_list.append(i)
        globals()["blnk_rmv_all".format(col)] = data.iloc[notna_list]
        print(globals()["blnk_rmv_all".format(col)].shape)
        print("OUT: blnk_rmv_all, blnk_rmv_{}")
    elif opt == 'col':
        data_cnt = len(data)
        notna_list = []
        col = ''.join(col)
        for i in range(data_cnt):
            if (data.loc[i,col] != ''):
                notna_list.append(i)
        globals()["blnk_rmv_{}".format(col)] = data.iloc[notna_list].copy()
        print(globals()["blnk_rmv_{}".format(col)].shape)
        print("OUT: blnk_rmv_all, blnk_rmv_{}")

In [3]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  "import pandas as pd\nimport numpy as np\nimport os\nimport string\nimport sys\nimport re\nimport datetime as dt\n# now = dt.datetime.now().strftime('%y%m%d_%H%M%S')\npd.set_option('display.max_rows', 500)\npd.set_option('display.max_columns', 500)",
  '# null/입력 값 검색 로직\n\ndef nll_chk(data):\n    cnt_chk = len(data)\n\n    chk_nll_list = []\n    chk_nll_cnt_list = []\n    # null값 체크(개수 파악)\n    for i,j in zip(range(len(data.columns)), data.columns):\n        indv_cnt = data.loc[data[j] != \'\', j]\n        if len(data[j]) != len(indv_cnt):\n            print(j + ": " + str(len(indv_cnt)))\n            chk_nll_list.append(j)\n            chk_nll_cnt_list.append(len(indv_cnt) - cnt_chk)\n    print("==============